In [14]:
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet

In [15]:
# code adapted from:
# https://github.com/FelixChop/MediumArticles/blob/master/LDA-BBC.ipynb

# load in data
data: pd.DataFrame = pd.read_csv('articlesData-ukraine.csv')
data = data.dropna().reset_index(drop=True)
data.shape
### FINISH THIS

(108, 3)

In [16]:
data.head

<bound method NDFrame.head of                                                  title  \
0                      DEC Ukraine Humanitarian Appeal   
1                                       War In Ukraine   
2              Ukraine: Reporting from the Front Lines   
3                                 The Ukraine Invasion   
4                                  Letter from Ukraine   
..                                                 ...   
103  Ukraine war: London cab driver leads convoy to...   
104  Ukraine: Aberdeen heating network terminates G...   
105  Ukraine daily roundup: World leaders show unit...   
106  Ukraine war: A dangerous escape on the 'Rescue...   
107                                      lastUpdated:    

                                               summary  \
0    An appeal on behalf of the DEC for the humanit...   
1    BBC News reports on the latest developments in...   
2    The BBC's teams report from the front lines on...   
3    Yalda Hakim is in Lviv with the late

In [17]:
# tokenise

data['sentences'] = data.title.progress_map(sent_tokenize)
data['sentences'].head(1).tolist()

data['tokens_sentences'] = data['sentences'].progress_map(
    lambda sentences: [word_tokenize(sentence)] for sentence in sentences)
data['POS_tokens'] = data['tokens_sentences'].progress_map(
    lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(data['POS_tokens'].head(1).tolist()[0][:3])

AttributeError: 'Series' object has no attribute 'progress_map'

In [ ]:
# Inspired from https://stackoverflow.com/a/15590384
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''


from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Lemmatizing each word with its POS tag, in each sentence
data['tokens_sentences_lemmatized'] = data['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1]))
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ]
        for tokens_POS in list_tokens_POS
    ]
)

data['tokens_sentences_lemmatized'].head(1).tolist()[0][:3]

# Regrouping tokens and removing stop words

from nltk.corpus import stopwords

stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use',
                   'would', 'can']
stopwords_other = ['one', 'mr', 'bbc', 'image', 'getty', 'de', 'en', 'caption', 'also', 'copyright', 'something']
my_stopwords = stopwords.words('English') + stopwords_verbs + stopwords_other

from itertools import chain  # to flatten list of sentences of tokens into list of tokens

data['tokens'] = data['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
data['tokens'] = data['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha()
                                                    and token.lower() not in my_stopwords and len(token) > 1])

data['tokens'].head(1).tolist()[0][:30]